# Assignment 8

Bill Henderson

CSCI-E-116

Spring 2025

## Setup
### Imports

In [2]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import root_mean_squared_error
from fredapi import Fred
import seaborn as sns
import yaml
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.var_model import VAR
import matplotlib.pyplot as plt
import warnings
    
warnings.simplefilter("ignore", category=UserWarning)
np.random.seed(42)

### Data Ingress

In [4]:
df = pd.read_excel('W09c_macro.xlsx', sheet_name=0)
df.columns = [x.lower() for x in df.columns]
df = df.set_index(pd.to_datetime(df.time)).drop('time',axis=1)

df

,ipmfg_ip,ip_ip,cumfg_ip,iptp_ip,ip54_ip,ip53_ip,ipfp_ip,ip521_ip,ip51_ip,ip511_ip,...,ypwgm_usna,saftod_usecon,safto_usecon,defenseshipments,cpg_usecon,statelocalemp,lafgda_labor,cpgf_usecon,cpgs_usecon,laptsva_usecon
time,,,,,,,,,,,,,,,,,,,,,
1990-01-31,61.3727,61.6352,81.4034,70.3538,76.3650,52.6478,68.1418,40.8392,78.9231,56.6829,...,3.919216,202.401700,763.460483,64.815709,830.352941,15048,742.1,2870.946342,21246.096366,4513.8
1990-02-28,62.2025,62.1951,82.2789,70.9906,76.3269,53.1280,68.9914,41.5484,79.8970,60.1486,...,3.926563,196.909886,826.381556,62.093341,852.343750,15068,739.6,2854.965818,21924.411327,4542.6
1990-03-31,62.5072,62.4916,82.4610,71.5156,76.6285,53.2146,69.5854,42.2517,80.4007,61.2557,...,3.951788,210.559390,861.155168,68.275221,819.930016,15100,733.0,2735.140995,21149.038873,4558.5
1990-04-30,62.2586,62.3511,81.9214,71.1807,76.3714,53.2505,69.2272,42.0032,80.0675,59.8669,...,3.978278,186.233663,772.514435,64.480681,820.837859,15118,727.6,2998.485862,20922.029487,4565.5
1990-05-31,62.4485,62.5353,81.9677,71.4049,76.5849,53.3954,69.4540,42.6040,80.0021,60.3753,...,4.039504,204.770546,864.540474,68.778237,824.175058,15164,725.5,2655.899071,21343.278594,4568.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,99.8849,103.2241,78.7116,101.5265,100.9500,105.2157,101.7750,95.7177,102.9887,107.2646,...,5.616569,209.313858,1469.121859,44.289052,1444.652710,19745,548.2,2344.443271,27666.404923,10767.1
2023-05-31,99.7784,102.9809,78.5451,101.1843,100.7935,105.0954,101.3571,95.6734,102.3006,108.2442,...,5.639082,225.339185,1733.204847,43.663574,1465.584362,19788,549.6,2287.612643,28082.306564,10810.4
2023-06-30,99.1514,102.3809,77.9690,100.3250,100.6211,104.8219,100.2180,95.5606,100.7068,104.9185,...,5.665405,260.849199,1952.982693,44.575218,1483.436019,19837,550.1,2199.975885,28513.940426,10831.7


In [5]:
quarterly = df.to_period('Q')
quarterly

,ipmfg_ip,ip_ip,cumfg_ip,iptp_ip,ip54_ip,ip53_ip,ipfp_ip,ip521_ip,ip51_ip,ip511_ip,...,ypwgm_usna,saftod_usecon,safto_usecon,defenseshipments,cpg_usecon,statelocalemp,lafgda_labor,cpgf_usecon,cpgs_usecon,laptsva_usecon
time,,,,,,,,,,,,,,,,,,,,,
1990Q1,61.3727,61.6352,81.4034,70.3538,76.3650,52.6478,68.1418,40.8392,78.9231,56.6829,...,3.919216,202.401700,763.460483,64.815709,830.352941,15048,742.1,2870.946342,21246.096366,4513.8
1990Q1,62.2025,62.1951,82.2789,70.9906,76.3269,53.1280,68.9914,41.5484,79.8970,60.1486,...,3.926563,196.909886,826.381556,62.093341,852.343750,15068,739.6,2854.965818,21924.411327,4542.6
1990Q1,62.5072,62.4916,82.4610,71.5156,76.6285,53.2146,69.5854,42.2517,80.4007,61.2557,...,3.951788,210.559390,861.155168,68.275221,819.930016,15100,733.0,2735.140995,21149.038873,4558.5
1990Q2,62.2586,62.3511,81.9214,71.1807,76.3714,53.2505,69.2272,42.0032,80.0675,59.8669,...,3.978278,186.233663,772.514435,64.480681,820.837859,15118,727.6,2998.485862,20922.029487,4565.5
1990Q2,62.4485,62.5353,81.9677,71.4049,76.5849,53.3954,69.4540,42.6040,80.0021,60.3753,...,4.039504,204.770546,864.540474,68.778237,824.175058,15164,725.5,2655.899071,21343.278594,4568.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023Q2,99.8849,103.2241,78.7116,101.5265,100.9500,105.2157,101.7750,95.7177,102.9887,107.2646,...,5.616569,209.313858,1469.121859,44.289052,1444.652710,19745,548.2,2344.443271,27666.404923,10767.1
2023Q2,99.7784,102.9809,78.5451,101.1843,100.7935,105.0954,101.3571,95.6734,102.3006,108.2442,...,5.639082,225.339185,1733.204847,43.663574,1465.584362,19788,549.6,2287.612643,28082.306564,10810.4
2023Q2,99.1514,102.3809,77.9690,100.3250,100.6211,104.8219,100.2180,95.5606,100.7068,104.9185,...,5.665405,260.849199,1952.982693,44.575218,1483.436019,19837,550.1,2199.975885,28513.940426,10831.7


In [9]:
fred_key = yaml.safe_load(open('/Users/hendwi1/.env'))['fred_key']

f_inf = Fred(api_key=fred_key)
gdpc = f_inf.get_series("GDPC1")

train_set = quarterly[quarterly.index <= "2023Q2"].copy()

test_set = gdpc.to_period('Q').loc["2023Q3":"2024Q2"]
train_set['gdpc1'] = gdpc.to_period('Q').loc[:"2023Q3"]

print("train_set",train_set.index.min(), train_set.index.max())
print("test_set",test_set.index.min(), test_set.index.max())

train_set 1990Q1 2023Q2
test_set 2023Q3 2024Q2


## Part A
### Dynamic Factor Model
#### VAR
3 PC's and 6 Lags

In [11]:
# setting up and scaling data
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_set)
pca = PCA(n_components=3)

pca.fit(train_scaled)

PCA(n_components=3)

In [13]:
transformed_data = pca.transform(train_scaled)

explained_variance = pca.explained_variance_ratio_
# Print the results
print("Original data:\n", train_set.head())
print("Scaled data:\n", train_scaled)
print("Transformed data:\n", transformed_data)
print("Explained variance ratio:", explained_variance)

Original data:
         ipmfg_ip    ip_ip  cumfg_ip  iptp_ip  ip54_ip  ip53_ip  ipfp_ip  \
time                                                                      
1990Q1   61.3727  61.6352   81.4034  70.3538  76.3650  52.6478  68.1418   
1990Q1   62.2025  62.1951   82.2789  70.9906  76.3269  53.1280  68.9914   
1990Q1   62.5072  62.4916   82.4610  71.5156  76.6285  53.2146  69.5854   
1990Q2   62.2586  62.3511   81.9214  71.1807  76.3714  53.2505  69.2272   
1990Q2   62.4485  62.5353   81.9677  71.4049  76.5849  53.3954  69.4540   

        ip521_ip  ip51_ip  ip511_ip  ...  saftod_usecon  safto_usecon  \
time                                 ...                                
1990Q1   40.8392  78.9231   56.6829  ...     202.401700    763.460483   
1990Q1   41.5484  79.8970   60.1486  ...     196.909886    826.381556   
1990Q1   42.2517  80.4007   61.2557  ...     210.559390    861.155168   
1990Q2   42.0032  80.0675   59.8669  ...     186.233663    772.514435   
1990Q2   42.6040  80

In [16]:
pca_df = pd.DataFrame(transformed_data, index = train_set.index)
pca_df.columns = [f"pca{i}" for i in pca_df.columns]

mod_var = VAR(pca_df)
res_var = mod_var.fit(maxlags = 6, trend="ct")
pc_fcst = res_var.forecast(mod_var.endog, steps=4)

pc_fcst = pd.DataFrame(pc_fcst, index = test_set.index, columns = pca_df.columns )

pc_fcst

,pca0,pca1,pca2
2023Q3,11.057913,0.259258,5.813836
2023Q4,10.996767,0.136524,5.941927
2024Q1,11.032075,0.110543,6.037421
2024Q2,11.055607,0.092773,6.011792


In [ ]:


rmse_dfm = root_mean_squared_error(test_set, fcst_dfm)
rmse_dfm

NameError: name 'fit01' is not defined

#### ARIMA